# fuzzy string matching
## 📚 resources
- `seatgeek` / [`thefuzz`](https://github.com/seatgeek/thefuzz) > README > [Usage](https://github.com/seatgeek/thefuzz/blob/83bea3d4a109a5d0c2e07334b504953cda4959c5/README.rst#Usage)
## details
- **to select, from one or more possible labels, the label that best matches the IRI slug**  
- *NOTE 🚩 that this is only enabled by the fact that ONS IRI slugs are created from labels with space, punctuation, etc. stripped*  


In [48]:
from some_ons import some_ons

In [49]:
from thefuzz import fuzz, process
from rdflib import Graph

In [50]:
with open("fuzzy_matching.md", 'w+') as mdfile:
    for entry in tuples_list:
        mdfile.write(f"# {entry[0]}\n")
        iri_slug = entry[0].split('/')[-1]
        mdfile.write(f"## Best match for iri_slug '{iri_slug}' ??\n")
        for item in entry[1::]:
            mdfile.write(f"### {item}\n")
            mdfile.write(f"- fuzz.ratio = {fuzz.ratio(iri_slug, item)}\n")
            mdfile.write(f"- fuzz.partial_ratio = {fuzz.partial_ratio(iri_slug, item)}\n")
            mdfile.write(f"- fuzz.token_sort_ration = {fuzz.token_sort_ratio(iri_slug, item)}\n")
            # to do -- remove parenthisized role information from creator display values prior to comparing
            # to do -- add *actual* rdfs:label from ONS
